In [7]:
import os
import json
import csv

# Define the directory containing the JSON files
directory = 'data'

# Define the output CSV file path
output_file = 'output.csv'

# Get a list of JSON files in the directory
json_files = [file for file in os.listdir(directory) if file.endswith('.json')]

# Open the output CSV file in write mode
with open(output_file, 'w', newline='') as csv_file:
    # Create a CSV writer
    fieldnames = ['uuid', 'item_id', 'timestamp', 'price', 'url', 'color', 'title']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Iterate over each JSON file
    for json_file in json_files:
        file_path = os.path.join(directory, json_file)
        
        # Open the JSON file and read its contents line by line
        with open(file_path) as file:
            for line in file:
                # Parse each line as a JSON object
                item = json.loads(line)
                if item['Item']['uuid']['S'] == 'maxtimestamp':
                    continue
                
                # Extract the relevant values from the item
                row = {
                    'uuid': item['Item']['uuid']['S'],
                    'item_id': item['Item']['item_id']['S'],
                    'timestamp': item['Item']['timestamp']['N'],
                    'price': item['Item']['price']['N'],
                    'url': item['Item']['url']['S'],
                    'color': item['Item']['color']['S'],
                    'title': item['Item']['title']['S']
                }
                
                # Write the row to the CSV file
                writer.writerow(row)

print(f"CSV file '{output_file}' created successfully.")

CSV file 'output.csv' created successfully.


In [6]:
import pandas as pd
from collections import deque

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('output.csv')

# Convert the 'timestamp' column to integer type
df['timestamp'] = df['timestamp'].astype(int)

# Sort the DataFrame by 'timestamp' in ascending order
df = df.sort_values('timestamp')

# Create a dictionary to store the latest occurrence of each item_id for each timestamp
latest_items = {}

# Initialize the timestamp queue with the first timestamp
timestamp_queue = deque([df['timestamp'].iloc[0]])

# Iterate over each row in the DataFrame and mark new items for each timestamp
def mark_new_items(row):
    item_id = row['item_id']
    timestamp = row['timestamp']

    if timestamp not in latest_items:
        latest_items[timestamp] = set()

    if timestamp_queue[0] != timestamp and timestamp not in timestamp_queue:
        timestamp_queue.append(timestamp)
        if len(timestamp_queue) > 2:
            timestamp_queue.popleft()
    
    print(timestamp_queue)

    prev_max_timestamp = timestamp_queue[0]

    if item_id not in latest_items[prev_max_timestamp]:
        row['is_new'] = '1'
        latest_items[timestamp].add(item_id)
    else:
        row['is_new'] = '0'

    return row

df = df.apply(mark_new_items, axis=1)

# Save the updated DataFrame back to the CSV file
df.to_csv('output_updated.csv', index=False)

print("CSV file updated with 'is_new' column.")

deque([1715889368])
deque([1715889368])
deque([1715889368])
deque([1715889368])
deque([1715889368])
deque([1715889368])
deque([1715889368, 1715889967])
deque([1715889368, 1715889967])
deque([1715889368, 1715889967])
deque([1715889368, 1715889967])
deque([1715889368, 1715889967])
deque([1715889368, 1715889967])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715889967, 1715920800])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715920800, 1715921167])
deque([1715921167, 1715921767])
deque([1715921167, 1715921767])
deque([1715921167, 1715921767])
deque([171592116

In [10]:
# Filter out rows where 'is_new' is 0
# import pandas as pd
# df = pd.read_csv('public/output_updated.csv')
df = df[df['is_new'] == '1']

# Save the filtered DataFrame back to the CSV file
df.to_csv('output_updated.csv', index=False)

print("CSV file updated to only include new items.")

# Convert timestamp to date and add it as a new column
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

# Drop duplicates based on 'item_id' and 'date'
df = df.drop_duplicates(subset=['item_id', 'date'])

# Save the updated DataFrame back to the CSV file
df.to_csv('output_updated.csv', index=False)

print("CSV file updated to remove duplicate item_ids with the same day.")


CSV file updated to only include new items.
CSV file updated to remove duplicate item_ids with the same day.


In [8]:
df[df['is_new'] == '1']

,uuid,item_id,timestamp,price,url,color,title,is_new
8049,H078595CKAA1715889368,H078595CKAA,1715889368,49705,/au/en/product/bolide-1923-45-racing-bag-H0785...,Blue,Bolide 1923 - 45 Racing bag,1
2682,H083894CAAG1715889368,H083894CAAG,1715889368,5900,/au/en/product/pursangle-tote-bag-H083894CAAG/,Multi-colored,Pursangle tote bag,1
8015,H082924CAAD1715889368,H082924CAAD,1715889368,11955,/au/en/product/cabas-h-en-biais-40-bag-H082924...,Multi-colored,Cabas H en Biais 40 bag,1
978,H083618CKAB1715889368,H083618CKAB,1715889368,7300,/au/en/product/steeple-25-bag-H083618CKAB/,Multi-colored,Steeple 25 bag,1
11574,H083618CKAC1715889368,H083618CKAC,1715889368,7300,/au/en/product/steeple-25-bag-H083618CKAC/,Multi-colored,Steeple 25 bag,1
...,...,...,...,...,...,...,...,...
9677,H085078CKAA1717490167,H085078CKAA,1717490167,9545,/au/en/product/garden-party-49-voyage-bag-H085...,Beige/Natural,Garden Party 49 voyage bag,1
5414,H082924CAAD1717490167,H082924CAAD,1717490167,11955,/au/en/product/cabas-h-en-biais-40-bag-H082924...,Multi-colored,Cabas H en Biais 40 bag,1
12321,H083189CKAO1717490767,H083189CKAO,1717490767,6560,/au/en/product/picotin-lock-18-pocket-bag-H083...,Orange,Picotin Lock 18 pocket bag,1
2898,H084235CC891717490767,H084235CC89,1717490767,14705,/au/en/product/picotin-lock-micro-bag-H084235C...,Black,Picotin Lock micro bag,1
